# EDA - Análisis Exploratorio de Datos para Rotación de Empleados

Análisis Exploratorio de Datos (EDA)
Este notebook explora el dataset de rotación de empleados de IBM para identificar patrones que expliquen la probabilidad de que un empleado abandone la empresa.


### Pregunta 1:
¿Qué impacto tiene el ingreso mensual (`MonthlyIncome`) combinado con el nivel del puesto (`JobLevel`) en la retención del personal?